In [1]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
driverLocation = "C:\\Users\leiwen\workspace\libs\chromedriver.exe"
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
import datetime as dt
from selenium.webdriver.support.select import Select

In [2]:
def mile_find(obj):
    
    obj["year"]=year
    try:
        obj["mile"] = obj["object"].find_element_by_xpath(".//span[contains(@class,'listing-row__mileage')]").text
    except:
        obj["mile"] = "unknown"
        
    try:
        obj["name"] = obj["object"].find_element_by_xpath(".//h2[contains(@class,'cui-delta listing-row__title')]").text
    except:
        obj["name"] = "unknown"

    try:
        obj["price"] = obj["object"].find_element_by_xpath(".//span[contains(@class,'listing-row__price')]").text
    except:
        obj["price"] = "unknown" 

    try:
        obj["hyperlink"] = obj["object"].find_element_by_xpath(".//a").get_attribute('href')
    except:
        obj["hyperlink"] = "unknown"

    try:
        obj["deal_qualilty"] = obj["object"].find_element_by_xpath(".//div[contains(@class,'listing-row__price-badge-arrow')]//span").text
    except:
        obj["deal_qualilty"] = "NOT A GOOD DEAL"

    try:
        attributes = obj["object"].find_elements_by_xpath(".//ul[contains(@class,'listing-row__meta')]//li")
        attribute = str()
        for i in attributes:
            attribute = attribute+ "," + i.text
            obj["other_attributes"] = attribute[1:]
    except:
        obj["other_attributes"] = "unknown"
    return obj

# external variables

In [3]:
body_style = "SUV"
condition = "Used Cars"
max_price = "$30,000"
zip_code= 22202
distance = "100"
year = "2008"

# internal variables

In [4]:
baseUrl = "https://www.cars.com/"
driver = webdriver.Chrome(driverLocation)
driver.get(baseUrl)
driver.maximize_window()

In [5]:
wait = WebDriverWait(driver, 20, poll_frequency=1,
                             ignored_exceptions=[NoSuchElementException,
                                                 ElementNotVisibleException,
                                                 ElementNotSelectableException])

In [6]:
data = pd.DataFrame()
total = 0
page_count = 1

# click and send key

In [7]:
cursor = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='search-by-body-style-tab']/label")))
cursor.click()

In [8]:
bsid = driver.find_element_by_xpath("//select[contains(@name,'bsId')]")
sel_id = Select(bsid)
sel_id.select_by_visible_text(body_style)

In [9]:
co = driver.find_element_by_xpath("//select[contains(@name,'stockType')]")
sel_co = Select(co)
sel_co.select_by_visible_text(condition)

In [10]:
max_ = driver.find_element_by_xpath("//select[contains(@name,'prMx')]")
sel_max = Select(max_)
sel_max.select_by_visible_text(max_price)

In [11]:
dis = driver.find_element_by_xpath("//select[contains(@name,'rd')]")
dis = Select(dis)
dis.select_by_value(distance)

In [12]:
zip_c = driver.find_element_by_xpath("//input[contains(@id,'zip')]")
zip_c.clear()
zip_c.send_keys(zip_code)

In [13]:
search = driver.find_element_by_xpath("//input[contains(@value,'Search')]")
search.click()

# landing search page now!

In [14]:
driver.implicitly_wait(3)

In [15]:
driver.execute_script("window.scrollTo(0, 50000)") 
page_show = wait.until(EC.presence_of_element_located((By.XPATH,"/html/body//cui-page-result-count//select")))
Select(page_show).select_by_value("100")

In [16]:
driver.implicitly_wait(3)

In [17]:
year1 = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='srpFilters']/ul/li[3]/div[2]/div[2]/select")))
year1 = Select(year1)
year1.select_by_visible_text(year)

In [18]:
time.sleep(3)

In [19]:
year2 = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='srpFilters']/ul/li[3]/div[2]/div[1]/select")))
year2 = Select(year2)
year2.select_by_visible_text(year)

# anchor tag disable:https://github.com/angular/protractor/issues/577

In [20]:
wait = WebDriverWait(driver, 20, poll_frequency=1,
                             ignored_exceptions=[NoSuchElementException,
                                                 ElementNotVisibleException,
                                                 ElementNotSelectableException])

In [21]:
while(True):
    elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH,"//div//div[@class='listing-row__details']")))
    temp = pd.DataFrame({'object':elements})
    #elements = driver.find_elements(By.XPATH,"//div//div[@class='listing-row__details']")
    length = len(elements)
    print("there are "+str(length) + " records in page "+ str(page_count)+" time:" + str(dt.datetime.now()))
    next = driver.find_element_by_xpath("//a[contains(text(),'Next')]")
    
    temp = temp.apply(lambda x : mile_find(x),axis = 1).iloc[:,1:]
    data = pd.concat([data,temp],axis = 0,ignore_index = True)
    total = total + length
    if next.get_attribute('disabled') == None:
        driver.execute_script("window.scrollTo(0, 50000)") 
        next.click()
        page_count = page_count +1
    else: 
        break

there are 100 records in page 1 time:2018-03-21 22:33:57.524235
there are 100 records in page 2 time:2018-03-21 22:34:35.591204
there are 100 records in page 3 time:2018-03-21 22:35:26.095692
there are 100 records in page 4 time:2018-03-21 22:36:13.977209
there are 100 records in page 5 time:2018-03-21 22:37:20.030029
there are 100 records in page 6 time:2018-03-21 22:38:28.188191
there are 100 records in page 7 time:2018-03-21 22:40:07.702271
there are 42 records in page 8 time:2018-03-21 22:42:19.529214


In [22]:
data

,year,mile,name,price,hyperlink,deal_qualilty,other_attributes
0,2008,"81,609 mi.",2008 BMW X5 4.8i,"$11,981",https://www.cars.com/vehicledetail/detail/7307...,GREAT DEAL,"Exterior Color: White Warranty,Interior Color:..."
1,2008,"73,136 mi.",2008 BMW X5 3.0si,"$11,750",https://www.cars.com/vehicledetail/detail/7299...,GREAT DEAL,"Exterior Color: Black,Transmission: 6-SPEED A/..."
2,2008,"188,335 mi.",2008 Nissan Pathfinder S,"$5,995",https://www.cars.com/vehicledetail/detail/7272...,GREAT DEAL,"Exterior Color: Majestic Blue Metallic,Interio..."
3,2008,"98,513 mi.",2008 Honda CR-V EX,"$9,800",https://www.cars.com/vehicledetail/detail/7299...,GOOD DEAL,"Exterior Color: Black,Transmission: 5-Speed Au..."
4,2008,"149,642 mi.",2008 Chevrolet Suburban 1500 LTZ,"$13,995",https://www.cars.com/vehicledetail/detail/7307...,GOOD DEAL,"Exterior Color: Blue,Interior Color: Black,Tra..."
5,2008,"104,460 mi.",2008 Mazda CX-9 Grand Touring,"$9,550",https://www.cars.com/vehicledetail/detail/7248...,GOOD DEAL,"Exterior Color: Liquid Platinum Metallic,Inter..."
6,2008,"117,288 mi.",2008 Jeep Liberty Sport,"$7,700",https://www.cars.com/vehicledetail/detail/7297...,GREAT DEAL,"Exterior Color: Modern Blue Pearl,Interior Col..."
7,2008,"97,957 mi.",2008 Honda CR-V EX,"$9,788",https://www.cars.com/vehicledetail/detail/7294...,NOT A GOOD DEAL,"Exterior Color: Red,Interior Color: Gray,Trans..."
8,2008,"77,092 mi.",2008 Honda CR-V EX,"$10,995",https://www.cars.com/vehicledetail/detail/7293...,NOT A GOOD DEAL,"Exterior Color: Red,Transmission: 5-Speed Auto..."
9,2008,"118,179 mi.",2008 Lexus RX 350,"$13,500",https://www.cars.com/vehicledetail/detail/7289...,GOOD DEAL,"Exterior Color: Desert Sage Metallic,Interior ..."


In [23]:
print("there are "+str(total) + " in total")

there are 742in total


In [24]:
data.to_csv(str(body_style)+" " +str(zip_code) +" "+ "less than " +str(distance) +" miles"+str(year) +" year.csv",index = False)